In [1]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [2]:
properties = pd.read_csv('trainPreProcess.csv')

In [3]:
#tomo pocos datos porque se hace muy lento el algoritmo si no (por ahora)
properties = properties.head(550)

In [4]:
properties.sample(5)

,Unnamed: 0,id,created_on,property_type_encoded,place_name_encoded,state_name_encoded,lat,lon,surface_covered_in_m2,surface_total_in_m2,floor,rooms,price,distancia_obelisco,distancia_minima_subte,distancia_minima_privado,distancia_minima_publico
155,194,396f0367a7fdfe85729a22133175e4d6904cf360,2013-05-24,2,644,1,-34.688727,-58.498525,86.0,110.0,0.0,2.0,56000.0,0.202523,0.082288,0.036831,0.074586
16,27,f252f0ddc642d1ff7d86c7daafff1aecefa7e87d,2017-03-24,1,452,0,-34.546018,-58.696385,51.5,0.0,0.0,2.0,454000.0,0.371789,0.238300,0.036831,0.074586
305,396,9623ad0971318aee87b06f390e1409a251cab0b0,2013-08-07,2,148,4,-34.560870,-58.460546,68.0,78.0,0.0,3.0,265000.0,0.121098,0.005496,0.036831,0.074586
303,394,103edc893dcabbeb14390bd23833cb4f24cab6d8,2013-08-07,2,631,4,-34.604376,-58.441855,46.0,53.0,0.0,2.0,146850.0,0.061503,0.007492,0.036831,0.074586
23,39,4dcfa9a54bdd18cd5438221c6dcfa94738c11c40,2013-02-15,2,295,0,-34.533846,-58.489214,111.0,115.0,0.0,4.0,275000.0,0.176790,0.043302,0.036831,0.074586


In [5]:
#quito unnamed:0 y created_on, y dejo price al final, id al principio
train = properties.loc[:, ['id', 'property_type_encoded', 'place_name_encoded', 'state_name_encoded', 'lat', 'lon', 'surface_covered_in_m2', 'surface_total_in_m2', 'floor', 'rooms', 'distancia_obelisco', 'distancia_minima_subte', 'distancia_minima_privado', 'distancia_minima_publico', 'price']]

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 15 columns):
id                          550 non-null object
property_type_encoded       550 non-null int64
place_name_encoded          550 non-null int64
state_name_encoded          550 non-null int64
lat                         550 non-null float64
lon                         550 non-null float64
surface_covered_in_m2       550 non-null float64
surface_total_in_m2         550 non-null float64
floor                       550 non-null float64
rooms                       550 non-null float64
distancia_obelisco          550 non-null float64
distancia_minima_subte      550 non-null float64
distancia_minima_privado    550 non-null float64
distancia_minima_publico    550 non-null float64
price                       550 non-null float64
dtypes: float64(11), int64(3), object(1)
memory usage: 64.5+ KB


In [7]:
#XGBoost funciona con matrices, no con Df
train = train.as_matrix()

In [8]:
#separo price
X = train[:,0:14]
Y = train[:,14]

In [9]:
#divido el set de datos en train y test
#marco el seed para que el RNG me de el mismo resultado
seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [10]:
#tomo los ids del test aparte para despues
x_test_ids = X_test[:,0]

In [11]:
#quito los ids de los sets, XGBoost solo trabaja con floats e ints
X_test = X_test[:,1:14]
X_train = X_train[:,1:14]

In [12]:
#armo el modelo (va a tomarse su tiempo con mas de 1000)
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [13]:
predictions = model.predict(X_test)

In [14]:
score = mean_squared_error(y_test, predictions)
score

26254761159.70303

In [15]:
median_absolute_error(y_test, predictions)

30000.0

In [16]:
#armo un csv con id, precio real, y precio predicho por XGBoost
predictionsflat = predictions.flatten()
pricesflat = y_test.flatten()
idsflat = x_test_ids.flatten()
submissionDf = pd.DataFrame()
submissionDf['id'] = pd.Series(idsflat)
submissionDf['real_price_usd'] = pd.Series(pricesflat)
submissionDf['XGB-predicted_price'] = pd.Series(predictionsflat)
submissionDf.set_index(['id'], inplace=True)
submissionDf.to_csv('predicciones_XGB.csv')

In [17]:
submissionDf.head(10)

,real_price_usd,XGB-predicted_price
id,,
07620eb32a6ac250879daf0e6d771bcb10087abb,280000,79500
c369f8ab5104df22245350d13788e39eb75873e0,650000,800000
cf7f335d2fc52fa5b26259fe0e430b68dec3e8fb,104000,107000
058002afb701eee91559b9f90b543403dd2c9a86,175900,60000
b8546c0ccc3ab7ace1af97e2f7a42a469d83e890,55000,70000
93dc054ec1f10731c67735422d59d5ff0a5edf53,190000,70000
e0453e27bdbe38c23c657198c5ca6d2ffdc5c36a,68000,55000
ecc8a2c6130331693ca35ccb74981ace12019b80,65000,68000
dee414c7198dd58137279f0afc56551481535cdc,500000,169954
